# Summary

-----

# Imports

In [1]:
NOTEBOOK_NAME = 'load_data'

In [2]:
%run imports.ipynb

elaspic_training_set
2016-06-09 23:52:08.456316


# Load data

## Setup

In [3]:
CONNECTION_STR

'mysql://strokach:@192.168.6.19:3306/elaspic_training_set'

In [4]:
db = datapkg.DataFrameToMySQL(CONNECTION_STR, NOTEBOOK_NAME, STG_SERVER_IP, echo=False)

In [5]:
# Combine experimental mutation sets with ELASPIC features
sql_query_template = """\
SELECT 

    p.*,

    us.uniprot_sequence,

    d.uniprot_domain_id,
    d.pfam_clan,
    t.alignment_identity,
    t.alignment_coverage,
    t.alignment_score,
    m.norm_dope,
    mut.stability_energy_wt, mut.stability_energy_mut,
    mut.physchem_wt, mut.physchem_wt_ownchain, mut.physchem_mut, mut.physchem_mut_ownchain, 
    mut.matrix_score, 
    mut.secondary_structure_wt, mut.solvent_accessibility_wt, 
    mut.secondary_structure_mut, mut.solvent_accessibility_mut, 
    mut.provean_score, mut.ddg

FROM ({subquery}) p
JOIN uniprot_kb.uniprot_sequence us using (uniprot_id)
JOIN elaspic.uniprot_domain d using (uniprot_id)
LEFT JOIN elaspic.uniprot_domain_template t using (uniprot_domain_id)
LEFT JOIN elaspic.uniprot_domain_model m using (uniprot_domain_id)
LEFT JOIN elaspic.uniprot_domain_mutation mut using (uniprot_id, uniprot_domain_id, mutation)

-- These two lines control whether we include mutations which fall outside domain
WHERE (model_domain_def is not NULL and elaspic.MUTATION_IN_DOMAIN(mutation, model_domain_def))
OR (model_domain_def is NULL and elaspic.MUTATION_IN_DOMAIN(mutation, domain_def));
"""

In [6]:
subqueries = dict()

subqueries['protein_folding_energy'] = """
SELECT
    uniprot_id,
    uniprot_mutation mutation,
    ddg_exp,
    null del_score_exp,
    null del_class_exp
FROM protein_folding_energy.{table_name}
WHERE ddg_exp is not null  -- remove mutations with no experimental ddG
AND (uniprot_mutation REGEXP '^[A-Za-z][0-9]+[A-Za-z]$')  -- remove weird mutations
AND (SUBSTRING(uniprot_mutation, 1, 1) != SUBSTRING(uniprot_mutation, -1))  -- remove synonymous mutations
"""

subqueries['protein_interface_energy'] = """
"""


subqueries['humsavar'] = """
SELECT
    uniprot_id,
    uniprot_mutation mutation,
    null ddg_exp,
    null del_score_exp,
    CASE type_of_variant WHEN 'Polymorphism' THEN 0 WHEN 'Disease' then 1 ELSE NULL END del_class_exp
FROM uniprot.humsavar
WHERE type_of_variant = 'Polymorphism' OR type_of_variant = 'Disease'
"""


subqueries['clinvar'] = """
SELECT
    us.uniprot_acc uniprot_id,
    d.mutationassessor_variant mutation,
    null ddg_exp,
    d.metalr_score del_score_exp,
    0 del_class_exp
FROM clinvar.clinvar_benign c
JOIN dbnsfp.variant d ON (c.id = d.rs_dbsnp146)
JOIN uniprot_kb_proteomes.UP000005640_9606_fasta us ON (us.uniprot_id = d.mutationassessor_uniprotid)
WHERE d.mutationassessor_variant IS NOT NULL
    UNION ALL
SELECT 
    us.uniprot_acc uniprot_id,
    d.mutationassessor_variant mutation,
    null ddg_exp,
    d.metalr_score del_score_exp,
    1 del_class_exp
FROM clinvar.clinvar c
JOIN dbnsfp.variant d ON (c.id = d.rs_dbsnp146)
JOIN uniprot_kb_proteomes.UP000005640_9606_fasta us ON (us.uniprot_id = d.mutationassessor_uniprotid)
WHERE d.mutationassessor_variant IS NOT NULL
"""



SELECT
    us.uniprot_acc uniprot_id,
    d.mutationassessor_variant mutation,
    null ddg_exp,
    d.metalr_score del_score_exp,
    0 del_class_exp
FROM clinvar.clinvar_benign c
JOIN dbnsfp.variant d ON (c.id = d.rs_dbsnp146)
JOIN uniprot_kb.uniprot_acc2uniprot_id us ON (us.uniprot_id = d.mutationassessor_uniprotid)
WHERE d.mutationassessor_variant IS NOT NULL
    UNION ALL
SELECT 
    us.uniprot_acc uniprot_id,
    d.mutationassessor_variant mutation,
    null ddg_exp,
    d.metalr_score del_score_exp,
    1 del_class_exp
FROM clinvar.clinvar c
JOIN dbnsfp.variant d ON (c.id = d.rs_dbsnp146)
JOIN uniprot_kb.uniprot_acc2uniprot_id us ON (us.uniprot_id = d.mutationassessor_uniprotid)
WHERE d.mutationassessor_variant IS NOT NULL






subqueries['cosmic'] = """
SELECT 
    us.uniprot_id, 
    d.mutationassessor_variant mutation,
    NULL ddg_exp,
    `FATHMM score` del_score_exp,
    CASE `FATHMM prediction` WHEN 'NEUTRAL' THEN 0 WHEN 'PATHOGENIC' THEN 1 ELSE NULL END del_class_exp
FROM cosmic.CosmicCodingMuts ccm
JOIN cosmic.CosmicMutantExport cme ON (cme.`Mutation ID` = ccm.ID)
JOIN dbnsfp.variant d ON (ccm.CHROM = d.chr AND ccm.POS = d.pos_1based AND ccm.REF = d.ref AND ccm.ALT = d.alt)
JOIN uniprot_kb.uniprot_sequence us ON (us.uniprot_name = d.mutationassessor_uniprotid)
WHERE us.uniprot_id not like '%-%' AND mutation IS NOT NULL AND `FATHMM prediction` IS NOT NULL;
"""

In [7]:
def load_dataset(table_name=None, engine=None):
    """
    """
    # Create sql query
    if table_name in ['humsavar', 'clinvar', 'cosmic']:
        subquery = subqueries[table_name]
    else:
        subquery = subqueries['protein_folding_energy'].format(table_name=table_name)
    sql_query = sql_query_template.format(subquery=subquery)
    
    # Read sql query
    df = pd.read_sql_query(sql_query, engine)
    df = df.rename(columns={'mutation': 'uniprot_mutation'})
    # df = ascommon.df_tools.remove_duplicate_columns(df)
    print("{}: ({})".format(table_name, df.shape[0]))
    
    # === All these rules could be converted to SQL... ===
    # Remove rows with null mutations
    null_mutants_ = (df['uniprot_mutation'].isnull()) | (df['uniprot_mutation'] == '-')
    print("Removing {} null mutants...".format(null_mutants_.sum()))
    df = df[~null_mutants_]

    # Remove SIFTS errors
    sifts_errors_ = df['uniprot_mutation'] == '?'
    print("Removing {} sifts errors...".format(sifts_errors_.sum()))
    df = df[~sifts_errors_]

    # Remove rows with weird mutations
    multi_mutants_ = df['uniprot_mutation'].str.contains(',')
    print("Removing {} multi mutants...".format(multi_mutants_.sum()))
    df = df[~multi_mutants_]

    # Remove wild-type rows
    wild_ = (df['uniprot_mutation'].str.lower() == 'wild')
    print("Removing {} wild mutants...".format(wild_.sum()))
    df = df[~wild_]

    # Remove synonymous mutations
    synonymous_ = df['uniprot_mutation'].str[0] == df['uniprot_mutation'].str[-1]
    print("Removing {} synonymous mutants...".format(synonymous_.sum()))
    df = df[~synonymous_]

    # Remove sequence mismatch mutations
    mutation_matches_sequence_ = (
        df[['uniprot_mutation', 'uniprot_sequence']]
        .apply(lambda x: ascommon.sequence_tools.mutation_matches_sequence(*x), axis=1)
    )
    mutation_matches_sequence_ = mutation_matches_sequence_.fillna(False)
    print("Removing {} mutations not matching sequence...".format((~mutation_matches_sequence_).sum()))
    df = df[mutation_matches_sequence_]

    # Convert ELASPIC features to expected format
    shape_before = df.shape[0]
    df = elaspic.elaspic_predictor.format_mutation_features(df, 'core')
    df = elaspic.elaspic_predictor.convert_features_to_differences(df)
    shape_after = df.shape[0]
    assert shape_before == shape_after
    
    # DONE!
    print('-' * 80)
    return df

## Run

In [8]:
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

### DATASETS

In [9]:
DATASETS = [
    'alascan_gpk',
    'curatedprotherm',
    'guerois',
    'kellogg',
    'potapov',
    'protherm',
    'taipale',
    'humsavar',
    'clinvar',
    'cosmic',
]

### DATA

In [10]:
DATA = {
    table_name: load_dataset(table_name=table_name, engine=db.engine)
    for table_name in DATASETS
}

alascan_gpk: (766)
Removing 0 null mutants...
Removing 0 sifts errors...
Removing 0 multi mutants...
Removing 0 wild mutants...
Removing 0 synonymous mutants...
Removing 3 mutations not matching sequence...
--------------------------------------------------------------------------------
curatedprotherm: (2923)
Removing 0 null mutants...
Removing 0 sifts errors...
Removing 0 multi mutants...
Removing 0 wild mutants...
Removing 0 synonymous mutants...
Removing 4 mutations not matching sequence...
--------------------------------------------------------------------------------
guerois: (939)
Removing 0 null mutants...
Removing 0 sifts errors...
Removing 0 multi mutants...
Removing 0 wild mutants...
Removing 0 synonymous mutants...
Removing 1 mutations not matching sequence...
--------------------------------------------------------------------------------
kellogg: (1145)
Removing 0 null mutants...
Removing 0 sifts errors...
Removing 0 multi mutants...
Removing 0 wild mutants...
Removing 0

KeyboardInterrupt: 

In [ ]:
with open(op.join(NOTEBOOK_NAME, 'DATA.pkl'), 'wb') as ofh:
    pickle.dump(DATA, ofh, pickle.HIGHEST_PROTOCOL)

### DF

In [ ]:
# In case of duplicates, the dataset occuring FIRST has the priority
df_list = []
for key in DATASETS:
    df = DATA[key]
    df['dataset'] = key
    df_list.append(df)
DF = pd.concat(df_list, ignore_index=True)

In [ ]:
print(DF.shape)

In [ ]:
DF = DF.drop_duplicates(subset=['uniprot_id', 'uniprot_mutation'], keep='first')
print(DF.shape)

In [ ]:
DF.to_pickle(op.join(NOTEBOOK_NAME, 'DF.pkl'))

### TRAINING_SETS

In [ ]:
TRAINING_SETS = {
    'protherm': DF[DF['dataset'].isin(
        {'alascan_gpk', 'curatedprotherm', 'guerois', 'kellogg', 'potapov', 'protherm'})],
    'taipale': DF[DF['dataset'] == 'taipale'],
    'humsavar': DF[DF['dataset'] == 'humsavar'],
}
assert sum(df.shape[0] for df in TRAINING_SETS.values()) == DF.shape[0]

In [ ]:
with open(op.join(NOTEBOOK_NAME, 'TRAINING_SETS.pkl'), 'wb') as ofh:
    pickle.dump(TRAINING_SETS, ofh, pickle.HIGHEST_PROTOCOL)